In [1]:
!pip install statsmodels scikit-learn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np

In [8]:
home_df = pd.read_csv('../Source/Metro_Home_Values.csv')
home_df.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,102001,0,United States,country,NaN,123951.733006,124163.176264,124425.524492,124988.958362,125637.500450,...,355986.448327,356640.365507,357118.576760,357576.035711,358418.372959,360011.602927,361969.362164,3.635033e+05,3.641659e+05,3.643848e+05
1,394913,1,"New York, NY",msa,NY,219657.598399,220586.011622,221519.660004,223407.993412,225353.501267,...,656116.650291,658992.330717,662322.088290,665541.460972,669925.455906,675684.524493,682290.159549,6.873056e+05,6.905686e+05,6.932481e+05
2,753899,2,"Los Angeles, CA",msa,CA,236649.976987,237553.116159,238728.814647,241071.926563,243614.826354,...,978656.081035,988006.132016,993962.586203,994044.629696,990763.189872,989599.749912,993777.833894,1.000601e+06,1.005222e+06,1.010107e+06


In [9]:
rental_df = pd.read_csv('../Source/Metro_Rentals.csv')
rental_df.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,102001,0,United States,country,NaN,1310.862067,1317.635432,1326.592064,1337.166174,1347.094223,...,2213.347989,2213.517002,2214.817776,2222.706032,2233.819378,2249.433872,2262.554936,2275.422072,2285.166408,2294.365189
1,394913,1,"New York, NY",msa,NY,2227.886183,2251.118894,2267.851695,2284.702372,2288.073225,...,3443.856725,3424.581955,3442.399810,3485.052220,3519.700761,3562.290444,3572.887480,3602.208908,3614.929134,3640.093145
2,753899,2,"Los Angeles, CA",msa,CA,2621.942240,2633.999717,2654.095188,2664.578607,2690.731090,...,4292.282261,4288.887028,4298.547999,4324.901797,4338.474621,4359.776105,4378.386198,4403.227148,4421.621816,4438.079122


In [10]:
unemployment_rate_df = pd.read_csv('../Source/UnemploymentRate.csv')
unemployment_rate_df.head(3)

,DATE,UNRATE
0,1/1/1948,3.4
1,2/1/1948,3.8
2,3/1/1948,4.0


In [17]:
# Read in mortgage rate data
mortgage_rate_df = pd.read_csv('../Source/MORTGAGE30US.csv')
mortage_rate_df.head(3)

,DATE,MORTGAGE30US
0,4/2/1971,7.33
1,4/9/1971,7.31
2,4/16/1971,7.31


In [12]:
med_hhi_df = pd.read_csv('../Source/MedianHHI.csv')
med_hhi_df.head(3)

,DATE,MEHOINUSA672N
0,1/1/1984,58930
1,1/1/1985,60050
2,1/1/1986,62280


In [18]:
# Convert 'DATE' column in unemployment_rate_df to datetime format
unemployment_rate_df['DATE'] = pd.to_datetime(unemployment_rate_df['DATE'], format='%m/%d/%Y')

# Convert 'DATE' column in mortage_rate_df to datetime format
mortage_rate_df['DATE'] = pd.to_datetime(mortage_rate_df['DATE'], format='%m/%d/%Y')

# Convert 'DATE' column in med_hhi_df to datetime format
med_hhi_df['DATE'] = pd.to_datetime(med_hhi_df['DATE'], format='%m/%d/%Y')

# For the home_df and rental_df, the columns may not have a DATE column directly.
# If the columns represent time as individual columns, you can combine them into a 'DATE' column.

# For home_df, suppose you are working with time series data as monthly columns:
# Melt the DataFrame to convert date columns to a 'DATE' column
home_df_melted = home_df.melt(id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], 
                              var_name='DATE', value_name='HomeValue')

# Convert 'DATE' to datetime format
home_df_melted['DATE'] = pd.to_datetime(home_df_melted['DATE'])

# For rental_df, apply the same process:
rental_df_melted = rental_df.melt(id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], 
                                  var_name='DATE', value_name='RentalValue')

# Convert 'DATE' to datetime format
rental_df_melted['DATE'] = pd.to_datetime(rental_df_melted['DATE'])

# Now you can merge all the data on the 'DATE' column.
# Example of merging the datasets together:
merged_df = pd.merge(home_df_melted, rental_df_melted, on=['RegionID', 'DATE'], how='inner')
merged_df = pd.merge(merged_df, unemployment_rate_df, on='DATE', how='inner')
merged_df = pd.merge(merged_df, mortage_rate_df, on='DATE', how='inner')
merged_df = pd.merge(merged_df, med_hhi_df, on='DATE', how='inner')

# Display the merged DataFrame
merged_df.head()

,RegionID,SizeRank_x,RegionName_x,RegionType_x,StateName_x,DATE,HomeValue,SizeRank_y,RegionName_y,RegionType_y,StateName_y,RentalValue,UNRATE,MORTGAGE30US,MEHOINUSA672N
